In [1]:
import pandas as pd
import warnings
import re
import tables
warnings.filterwarnings('ignore')

# Lendo planilhas

In [2]:
df_animais_sem_id = pd.read_hdf('df_animais_sem_id.h5', key='df')
df_genealogia_embrapa = pd.read_hdf('df_genealogia_embrapa.h5', key='df')
df_fazendas = pd.read_hdf('df_fazendas.h5', key='df')
df_criadores = pd.read_hdf('df_criadores.h5', key='df')
pedigree = pd.read_csv("pedrigree - Girolando - 17-02-2023.txt", encoding='ISO-8859-1')

# Imprimindo dataframes

In [3]:
df_animais_sem_id = df_animais_sem_id.drop_duplicates()
df_animais_sem_id = df_animais_sem_id.drop(columns='id_produto_x')
df_animais_sem_id = df_animais_sem_id.drop(columns='id_produto_y')
#df_animais_sem_id

# Criando dataframe df_animais_sem_id_completo pela juncao do df_animais_sem_id com pedigree
Obs: Mantendo somente o primeiro registro dos animais com IDANIMAL duplicados

In [4]:
df_animais_sem_id_completo = pd.merge(df_animais_sem_id, pedigree, on=['IDANIMAL'], how='left', indicator=True, suffixes=('','_drop'))
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(df_animais_sem_id_completo.filter(regex='_drop$').columns, axis=1)
df_animais_sem_id_completo = df_animais_sem_id_completo[df_animais_sem_id_completo['_merge']=='both']
df_animais_sem_id_completo = df_animais_sem_id_completo.drop_duplicates(subset='IDANIMAL')
#df_animais_sem_id_completo

# Criando id_produto dos animais que não possuem id_produto
obs: id_produto = giro+IDANIMAL

In [5]:
df_animais_sem_id_completo['id_produto'] = df_animais_sem_id_completo['id_produto'].fillna('GIRO' + df_animais_sem_id_completo['IDANIMAL'].astype(str))
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='_merge')
#df_animais_sem_id_completo

# Juntando df_animais_sem_id_completo (idpai) com genealogia_embrapa (idanimal) tabela separada para pedigree

In [6]:
df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, df_genealogia_embrapa[['id_produto', 'IDANIMAL']],left_on=['IDPAI'], how='left', right_on=['IDANIMAL'], suffixes=('', '_drop'))
df_animais_sem_id_completo = df_animais_sem_id_completo.drop_duplicates()
#df_animais_sem_id_completo

# Juntando df_animais_sem_id_completo (idmae) com genealogia_embrapa (idanimal)

In [7]:
df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, df_genealogia_embrapa[['id_produto', 'IDANIMAL']],left_on=['IDMAE'], how='left', right_on=['IDANIMAL'], suffixes=('', '_tes'))
df_animais_sem_id_completo = df_animais_sem_id_completo.drop_duplicates()
#df_animais_sem_id_completo

# Adicionando coluna IDTOURO e id_vaca no df_animais_sem_id_completo

In [8]:
df_animais_sem_id_completo = df_animais_sem_id_completo.rename(columns={'id_produto_drop':'id_touro'})
df_animais_sem_id_completo = df_animais_sem_id_completo.rename(columns={'id_produto_tes':'id_vaca'})
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='IDANIMAL_drop')
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='IDANIMAL_tes')
#df_animais_sem_id_completo

# Buscando o id_touro pelo cod_pai na genealogia_embrapa

In [9]:
df_id_touro = pd.merge(df_animais_sem_id_completo, df_genealogia_embrapa[['id_produto', 'cod_prod']], left_on=['cod_pai'], right_on=['cod_prod'], suffixes=('', '_drop'))
#df_id_touro

# Adicionando o id_touro no dataframe com todas as colunas pelo id_animal

In [10]:
df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, df_id_touro[['IDANIMAL','id_produto_drop']], on=['IDANIMAL'], how='left')
df_animais_sem_id_completo = df_animais_sem_id_completo.drop_duplicates()
df_animais_sem_id_completo['id_touro'].fillna(df_animais_sem_id_completo['id_produto_drop'], inplace=True)
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='id_produto_drop')
#df_animais_sem_id_completo

# Buscando o id_vaca pelo cod_vaca na genealogia_embrapa

In [11]:
df_id_vaca = pd.merge(df_animais_sem_id_completo, df_genealogia_embrapa[['id_produto', 'cod_prod']], left_on=['cod_mae'], right_on=['cod_prod'], indicator=True, suffixes=('', '_drop'))
#df_id_vaca

# Adicionando id_vaca no dataframe com todas as colunas

In [12]:
df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, df_id_vaca[['IDANIMAL','id_produto_drop']], on=['IDANIMAL'], how='left')
df_animais_sem_id_completo = df_animais_sem_id_completo.drop_duplicates()
df_animais_sem_id_completo['id_vaca'].fillna(df_animais_sem_id_completo['id_produto_drop'], inplace=True)
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='id_produto_drop')
#df_animais_sem_id_completo

# Pegando o id_produto que vira id_touro dos touros_gir na genealogia embrapa, em que o registro comeca com 3 letras

In [13]:
# somente gs_pai=11 e gs = 0 na genealogia
df_sub_tabela_gir = df_animais_sem_id_completo[df_animais_sem_id_completo['GSPAI']==11]
df_sub_tabela_gir = df_sub_tabela_gir[df_sub_tabela_gir['RGPAI'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
#df_sub_tabela_gir

# Pegando o id_produto que vira id_vaca das vacas_gir na genealogia embrapa, em que o registro comeca com 3 letras

In [14]:
# somente gs_pai=11 e gs = 0 na genealogia
df_sub_tabela_gir1 = df_animais_sem_id_completo[df_animais_sem_id_completo['GSMAE']==11]
df_sub_tabela_gir1 = df_sub_tabela_gir1[df_sub_tabela_gir1['RGMAE'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
#df_sub_tabela_gir1

# Juntando a genealogia embrapa com os touros comecando com 3 letras e gs 11

In [15]:
df_sub_touros_id = pd.merge(df_sub_tabela_gir, df_genealogia_embrapa[['id_produto','registro']], left_on=['RGPAI'], right_on=['registro'])
#df_sub_touros_id

# Juntando a genealogia embrapa com as vacas comecando com 3 letras e gs 11

In [16]:
df_sub_vacas_id = pd.merge(df_sub_tabela_gir1, df_genealogia_embrapa[['id_produto','registro']], left_on=['RGMAE'], right_on=['registro'])
#df_sub_vacas_id

In [17]:
df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, df_sub_touros_id[['IDANIMAL', 'id_produto_y']], on=['IDANIMAL'], how='left')
df_animais_sem_id_completo['id_touro'].fillna(df_animais_sem_id_completo['id_produto_y'], inplace=True)
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='id_produto_y')
df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, df_sub_vacas_id[['IDANIMAL', 'id_produto_y']], on=['IDANIMAL'], how='left')
df_animais_sem_id_completo['id_vaca'].fillna(df_animais_sem_id_completo['id_produto_y'], inplace=True)
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='id_produto_y')
#df_animais_sem_id_completo

Juntar codigo acima com genealogia
RGPAI(df_sub_tabela_gir) liga com registro(genealogia)
duplicar solucao para mae
RGPAI RGMAE RGVACA GS=11 retirar acentos e espacos

# Pegando os id_produto's dos touros_gir na genealogia embrapa, em que o registro nao comeca com 3 letras

In [18]:
df_sub_tabela_gir1 = df_animais_sem_id_completo[df_animais_sem_id_completo['GSPAI']==11]
df_sub_tabela_gir1 = df_sub_tabela_gir1[~df_sub_tabela_gir1['RGPAI'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
#Pegando os 4 primeiros caracteres do nome
df_sub_tabela_gir1['nome4caracteres'] = df_sub_tabela_gir1['nomep'].str.slice(start=0, stop=4)


df_subtabela_genealogia = df_genealogia_embrapa
df_subtabela_genealogia = df_subtabela_genealogia[df_subtabela_genealogia['gs']==0]
df_subtabela_genealogia = df_subtabela_genealogia[df_subtabela_genealogia['sexo']=='M']
df_subtabela_genealogia = df_subtabela_genealogia[~df_subtabela_genealogia['registro'].isnull()]
df_subtabela_genealogia = df_subtabela_genealogia[~df_subtabela_genealogia['nome'].isnull()]
df_subtabela_genealogia = df_subtabela_genealogia[~df_subtabela_genealogia['registro'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
df_subtabela_genealogia['nome4caracteres'] = df_subtabela_genealogia['nome'].str.slice(start=0, stop=4)

dj_juntando_tabelas = pd.merge(df_sub_tabela_gir1, df_subtabela_genealogia[['id_produto', 'registro', 'nome4caracteres']], left_on=['nome4caracteres', 'RGPAI'], right_on=['nome4caracteres', 'registro'])
#dj_juntando_tabelas

# Pegando os id_produto’s das vacas na genealogia embrapa, em que o registro nao comeca com 3 letras

In [19]:
df_sub_tabela_gir2 = df_animais_sem_id_completo[df_animais_sem_id_completo['GSMAE']==11]
df_sub_tabela_gir2 = df_sub_tabela_gir2[~df_sub_tabela_gir2['RGMAE'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
#Pegando os 4 primeiros caracteres do nome
df_sub_tabela_gir2['nome4caracteres'] = df_sub_tabela_gir2['nomem'].str.slice(start=0, stop=4)


df_subtabela_genealogia1 = df_genealogia_embrapa
df_subtabela_genealogia1 = df_subtabela_genealogia1[df_subtabela_genealogia1['gs']==0]
df_subtabela_genealogia1 = df_subtabela_genealogia1[df_subtabela_genealogia1['sexo']=='F']
df_subtabela_genealogia1 = df_subtabela_genealogia1[~df_subtabela_genealogia1['registro'].isnull()]
df_subtabela_genealogia1 = df_subtabela_genealogia1[~df_subtabela_genealogia1['nome'].isnull()]
df_subtabela_genealogia1 = df_subtabela_genealogia1[~df_subtabela_genealogia1['registro'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
df_subtabela_genealogia1['nome4caracteres'] = df_subtabela_genealogia1['nome'].str.slice(start=0, stop=4)

dj_juntando_tabelas1 = pd.merge(df_sub_tabela_gir2, df_subtabela_genealogia1[['id_produto', 'registro', 'nome4caracteres']], left_on=['nome4caracteres', 'RGMAE'], right_on=['nome4caracteres', 'registro'])
#dj_juntando_tabelas1

# Adicionando ID's touros e ID's vacas encontrados em que o registro não comeca com 3 letras na df_animais_sem_id_completo

In [20]:
df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, dj_juntando_tabelas[['IDANIMAL', 'id_produto_y']], on=['IDANIMAL'], how='left')
df_animais_sem_id_completo['id_touro'].fillna(df_animais_sem_id_completo['id_produto_y'], inplace=True)
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='id_produto_y')
df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, dj_juntando_tabelas1[['IDANIMAL', 'id_produto_y']], on=['IDANIMAL'], how='left')
df_animais_sem_id_completo['id_vaca'].fillna(df_animais_sem_id_completo['id_produto_y'], inplace=True)
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='id_produto_y')
#df_animais_sem_id_completo

# Lista de animais com id_vaca E id_touro

In [21]:
df_animais_com_ids = df_animais_sem_id_completo[~df_animais_sem_id_completo['id_vaca'].isnull() & ~df_animais_sem_id_completo['id_touro'].isnull()]
#df_animais_com_ids

# Lista de animais sem id_touro e sem id_vaca

In [22]:
df_animais_sem_nenhum_ids = df_animais_sem_id_completo[df_animais_sem_id_completo['id_vaca'].isnull() & df_animais_sem_id_completo['id_touro'].isnull()]
df_animais_sem_nenhum_ids

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,AN,NOMEVACA,DNASC,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,id_touro,id_vaca
14,8494,WILY 1151 DA JCS C.BONITA,38,3468-BJ,0,0,0,0,0,0,...,2019,WILY 1151 DA JCS C.BONITA,04/02/2019,F,NaN,NaN,3/8HOL 5/8GIR,417,NaN,NaN
35,5192,LIGIA FIV HUMBLENKIND 2403 RS,34,0578-BG,COOKIECUTTER HUMBLENKIND-ET,10,AX152772,KIRIA FIV ALAMBARI,12,5885-AT,...,2019,LIGIA FIV HUMBLENKIND 2403 RS DO RANCHO ALEGRE,30/04/2019,F,COOKIECUTTER HUMBLENKIND-ET,KIRIA FIV ALAMBARI,3/4HOL 1/4GIR,795,NaN,NaN
38,8494,VAIDOSA 1108 DA JCS C.BONITA,14,3470-BJ,0,0,0,0,0,0,...,2018,VAIDOSA 1108 DA JCS C.BONITA,18/08/2018,F,NaN,NaN,1/4HOL 3/4GIR,315,NaN,NaN
54,10032,ELITE RECANTO DO COQUEIRO,58,8014-BH,0,0,0,0,0,0,...,2019,ELITE RECANTO DO COQUEIRO,25/08/2019,F,NaN,NaN,5/8HOL 3/8GIR,693,NaN,NaN
65,10032,BELEZA RECANTO DO COQUEIRO,12,3748-AS,0,0,0,0,0,0,...,2014,BELEZA RECANTO DO COQUEIRO,10/09/2014,F,NaN,NaN,1/2HOL 1/2GIR,519,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,1026,ARTEMIS FBS CAETANO,12,6396-AR,0,0,0,0,0,0,...,2013,ARTEMIS FBS CAETANO,22/10/2013,F,NaN,NaN,1/2HOL 1/2GIR,519,NaN,NaN
528,5191,ORGANICA 5/8-591 2K,34,3860-BD,CACIQUE A. JORDAN NOVA TERRA,58,9140-M,GRINALDA 2K,78,6223-AA,...,2018,ORGANICA 5/8-591 2K,03/04/2018,F,CACIQUE A. JORDAN NOVA TERRA,GRINALDA 2K,3/4HOL 1/4GIR,795,NaN,NaN
530,10032,SERRA NEGRA DA SANQUIT,12,5043-AP,0,0,0,0,0,0,...,2013,SERRA NEGRA DA SANQUIT,31/05/2013,F,NaN,NaN,1/2HOL 1/2GIR,519,NaN,NaN
542,1129,UARA ANGICO,34,8692-AX,0,0,0,0,0,0,...,2017,UARA ANGICO,03/02/2017,F,NaN,NaN,3/4HOL 1/4GIR,795,NaN,NaN


# Lista de animais sem id_vaca

In [23]:
df_animais_sem_id_vaca = df_animais_sem_id_completo[df_animais_sem_id_completo['id_vaca'].isnull()]
df_animais_sem_id_vaca

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,AN,NOMEVACA,DNASC,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,id_touro,id_vaca
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,2015,NANA IMPERADOR BAXTER RF SAO FRANCISCO,11/03/2015,F,IMPERADOR BAXTER VOLTA FRIA,JURITI HERBY RF SAO FRANCISCO,5/8HOL 3/8GIR,666,GIRO977401,NaN
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,2013,EDILAINE IMPERIO FIV 4303 JARDIM DO TURVO,18/10/2013,F,IMPERIO TE STA EDWIGES,SERTEC QATAR FABIANA PROGRESS,1/2HOL 1/2GIR,519,RIG163,NaN
5,10586,JHV CANASTRA ENLEVO,12,5011-BG,ENLEVO SILVANIA,11,EFC717,JHV FORMATURA 315,10,C-3602,...,2019,JHV CANASTRA ENLEVO,18/06/2019,F,ENLEVO SILVANIA,JHV FORMATURA 315,1/2HOL 1/2GIR,519,EFC717,NaN
8,22541,KELLY ESPELHO JF DA SANTA RITA,12,7974-AM,ESPELHO TE DE BRASILIA,11,RRP5664,RACKELY JF DA SANTA RITA,10,A-7744,...,2012,KELLY ESPELHO JF DA SANTA RITA,06/04/2012,F,ESPELHO TE DE BRASILIA,RACKELY JF DA SANTA RITA,1/2HOL 1/2GIR,519,RRP5664,NaN
10,4406,VITORIA WINDSTAR IRARA II,34,6762-B,DUPASQUIER WINDSTAR,10,AX101932,SANDI IRARA II,12,X-2396,...,2009,VITORIA WINDSTAR IRARA II,18/05/2009,F,DUPASQUIER WINDSTAR,SANDI IRARA II,3/4HOL 1/4GIR,795,143607,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,9330,PALMADA CAL,14,0383-BN,DIORITO CAL,11,CAL9630,ILHOA FIV CAL,12,7229-AG,...,2020,PALMADA CAL,22/02/2020,F,DIORITO CAL,ILHOA FIV CAL,1/4HOL 3/4GIR,315,CAL9630,NaN
548,6029,GRAVINA FEBO FR RECREIO,5858,9073-AS,FEBO FR RECREIO,58,6300-N,DORINA FR RECREIO,58,0860-T,...,2016,GRAVINA FEBO FR RECREIO,13/06/2016,F,FEBO FR RECREIO,DORINA FR RECREIO,5/8HOL 3/8GIR,666,GIRO1114289,NaN
553,10032,CATUABA F.DA BARRA,12,2720-BM,0,0,0,0,0,0,...,2015,CATUABA F.DA BARRA,01/09/2015,F,NaN,NaN,1/2HOL 1/2GIR,519,NaN,NaN
557,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,2019,LIA DAS ARABIAS,13/06/2019,F,TURBO DAS ARABIAS,LILIKA DAS ARABIAS,5/8HOL 3/8GIR,666,GIRO1155542,NaN


# Lista de animais sem id_touro

In [24]:
df_animais_sem_id_touro = df_animais_sem_id_completo[df_animais_sem_id_completo['id_touro'].isnull()]
df_animais_sem_id_touro

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,AN,NOMEVACA,DNASC,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,id_touro,id_vaca
9,10586,MORANGA OPINIOSO DO SEGREDO VE,58,6930-BC,ARATINGA OPINIOSO 81 BROKAW TE,10,AX144696,SALINA NOBRE DO MORRO,14,C-4972,...,2018,MORANGA OPINIOSO DO SEGREDO VELHO,07/08/2018,F,ARATINGA OPINIOSO 81 BROKAW TE,SALINA NOBRE DO MORRO,5/8HOL 3/8GIR,693,NaN,GIRO821357
14,8494,WILY 1151 DA JCS C.BONITA,38,3468-BJ,0,0,0,0,0,0,...,2019,WILY 1151 DA JCS C.BONITA,04/02/2019,F,NaN,NaN,3/8HOL 5/8GIR,417,NaN,NaN
15,4237,JADI DYNAMO FIV FUNDAO,12,3640-BK,MR RUBICON DYNAMO-ET,10,AX152161,DIAMANTINA FIV ZBR,11,ZBR157,...,2019,JADI DYNAMO FIV FUNDAO,14/09/2019,F,MR RUBICON DYNAMO-ET,DIAMANTINA FIV ZBR,1/2HOL 1/2GIR,519,NaN,ZBR157
16,5569,CLAIRE UTOPIA LAMBDA SH FIV 2B,12,9276-BK,FARNEAR DELTA-LAMBDA-ET,10,AX152152,UTOPIA FIV 2B,11,ZAB1116,...,2019,CLAIRE UTOPIA LAMBDA SH FIV 2B,02/10/2019,F,FARNEAR DELTA-LAMBDA-ET,UTOPIA FIV 2B,1/2HOL 1/2GIR,519,NaN,ZAB1116
21,52717,TANISHA FIV MORENO ALEGRE,58,4758-BB,PEAK ALTAMORENO-ET,10,AX150663,PRATEADA FIV SANSAO JGVA,14,3599-F,...,2018,TANISHA FIV MORENO ALEGRE,19/03/2018,F,PEAK ALTAMORENO-ET,PRATEADA FIV SANSAO JGVA,5/8HOL 3/8GIR,693,NaN,GIRO984858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,5191,ORGANICA 5/8-591 2K,34,3860-BD,CACIQUE A. JORDAN NOVA TERRA,58,9140-M,GRINALDA 2K,78,6223-AA,...,2018,ORGANICA 5/8-591 2K,03/04/2018,F,CACIQUE A. JORDAN NOVA TERRA,GRINALDA 2K,3/4HOL 1/4GIR,795,NaN,NaN
530,10032,SERRA NEGRA DA SANQUIT,12,5043-AP,0,0,0,0,0,0,...,2013,SERRA NEGRA DA SANQUIT,31/05/2013,F,NaN,NaN,1/2HOL 1/2GIR,519,NaN,NaN
542,1129,UARA ANGICO,34,8692-AX,0,0,0,0,0,0,...,2017,UARA ANGICO,03/02/2017,F,NaN,NaN,3/4HOL 1/4GIR,795,NaN,NaN
553,10032,CATUABA F.DA BARRA,12,2720-BM,0,0,0,0,0,0,...,2015,CATUABA F.DA BARRA,01/09/2015,F,NaN,NaN,1/2HOL 1/2GIR,519,NaN,NaN


# Separando touros

In [25]:
df_animais_sem_id_touro_gir = df_animais_sem_id_touro[df_animais_sem_id_touro['GSPAI']==11]
#df_animais_sem_id_touro_gir

In [26]:
df_animais_sem_id_touro = df_animais_sem_id_touro[df_animais_sem_id_touro['GSPAI']!=11]
#df_animais_sem_id_touro

In [27]:
df_animais_com_ids['cgen_a']

1      1/2HOL 1/2GIR
3      1/2HOL 1/2GIR
4      1/2HOL 1/2GIR
6      1/2HOL 1/2GIR
7      1/2HOL 1/2GIR
           ...      
550    5/8HOL 3/8GIR
551    3/4HOL 1/4GIR
552    1/2HOL 1/2GIR
554    1/2HOL 1/2GIR
555    1/2HOL 1/2GIR
Name: cgen_a, Length: 303, dtype: object

# Montando tabela pro banco de df_animais_com_ids
OBS: Falta juntar com a genealogia embrapa(todas as colunas) e efetuar drop (ja feito) para a limpeza

In [28]:
teste = pd.merge(df_animais_com_ids, df_genealogia_embrapa, on=['cod_prod'], suffixes=('', '_drop'))
df_animais_com_ids

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,AN,NOMEVACA,DNASC,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,id_touro,id_vaca
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,2014,LANCA FIV AFTERSHOCK BON.BELEM,22/12/2014,F,MS ATLEES SHT AFTERSHOCK - ET,JOANE FIV KUBERA,1/2HOL 1/2GIR,519,207142,ACFG1840
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,2015,ARMA MERIDIAN DA MBL,27/04/2015,F,SULLY HART MERIDIAN-ET,FAMA,1/2HOL 1/2GIR,519,GIRO1369768,JNR167
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,2016,CELESTE 136 TEATRO FIV COOPERRITA,05/02/2016,F,TEATRO DA SILVANIA,LAGOS ASTRE QUIMICA 795 TE,1/2HOL 1/2GIR,519,EFC383,GIRO1086826
6,83436,PURA FIV DOORMAN SJ LALU,12,2678-BK,VAL-BISSON DOORMAN,10,AX140558,KANETA DP,11,DPJ955,...,2019,PURA FIV DOORMAN SJ LALU,29/08/2019,F,VAL-BISSON DOORMAN,KANETA DP,1/2HOL 1/2GIR,519,GIRO1407888,DPJ955
7,2381,GOTA CHELIOS JM NOVO HORIZONTE,12,7006-BC,DOMICOLE CHELIOS,10,AX131313,DALIA NOVO HORIZONTE,11,JMCH76,...,2018,GOTA CHELIOS JM NOVO HORIZONTE,13/06/2018,F,DOMICOLE CHELIOS,DALIA NOVO HORIZONTE,1/2HOL 1/2GIR,519,GIRO1336133,JMCH76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,8490,REUNIDAS HD FILADELFIA 17237 F,58,0122-BB,VIEW-HOME MONTEREY-ET,10,AX143697,REUNIDAS HD BAGDA 14426 CASTEL,14,8010-L,...,2018,REUNIDAS HD FILADELFIA 17237 FIV MONTEREY,31/07/2018,F,VIEW-HOME MONTEREY-ET,REUNIDAS HD BAGDA 14426 CASTEL,5/8HOL 3/8GIR,693,GIRO1531445,GIRO1265445
551,5426,SHRTC ESPERANAA AUSTRIA GOLD C,34,9428-AQ,MR CHASSITY GOLD CHIP-ET,10,AX135713,SHRTC AUSTRIA TEATRO FIV,12,8739-X,...,2016,SHRTC ESPERANCA AUSTRIA GOLD CHIP FIV 497,22/04/2016,F,MR CHASSITY GOLD CHIP-ET,SHRTC AUSTRIA TEATRO FIV,3/4HOL 1/4GIR,795,210297,GIRO1191848
552,2381,ESTIVA QUIZAR JM NOVO HORIZONT,12,9958-AS,COLLEM FEVER QUIZAR,10,AX134708,SC MANCHETE PALADINO,11,MJJS411,...,2016,ESTIVA QUIZAR JM NOVO HORIZONTE,16/05/2016,F,COLLEM FEVER QUIZAR,SC MANCHETE PALADINO,1/2HOL 1/2GIR,519,GIRO1481509,MJJS411
554,1945,JPZ QUIRERA SHAMROCK ASTER FIV,12,4032-Z,LADYS-MANOR PL SHAMROCK-ET,10,AX135714,JPZ ASTER FIV,11,JPZ64,...,2014,JPZ QUIRERA SHAMROCK ASTER FIV,18/02/2014,F,LADYS-MANOR PL SHAMROCK-ET,JPZ ASTER FIV,1/2HOL 1/2GIR,519,207140,JPZ64


Tabela de fazendas pegar o rebc2_pr -> rebc2-or (fazendas) <- reb_giro(df_animais_com_id)

ABCZ - S = Quando cod prod > 0 ABCZ
cat cod_prod >0 (5 comprac 11) ABCZ

id = id_produto
nas e nasc_est = DNASC


In [29]:
# Pegando rebc2_pr da tabela de fazendas
df_animais_com_ids = pd.merge(df_animais_com_ids, df_criadores[['rebc_or', 'codgiro']], left_on=['reb_giro'], right_on=['codgiro'])

df_animais_com_ids

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,DNASC,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,id_touro,id_vaca,rebc_or,codgiro
0,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,22/12/2014,F,MS ATLEES SHT AFTERSHOCK - ET,JOANE FIV KUBERA,1/2HOL 1/2GIR,519,207142,ACFG1840,10639-GIRO,10639
1,10639,FLORENTINA FIV ATWOOD SAMPA,12,0949-AK,MAPLE-DOWNS-I G W ATWOOD-ET,10,AX130581,JULIA FIV KUBERA,11,ACFG1848,...,05/08/2014,F,MAPLE-DOWNS-I G W ATWOOD-ET,JULIA FIV KUBERA,1/2HOL 1/2GIR,519,208574,ACFG1848,10639-GIRO,10639
2,10639,JGSA 1216 FIV GILCREST PORTEIR,12,9419-BI,DE-SU ALTAGILCREST-ET,10,AX141066,HERDEIRA,11,ERD137,...,08/08/2018,F,DE-SU ALTAGILCREST-ET,HERDEIRA,1/2HOL 1/2GIR,519,GIRO1455287,ERD137,10639-GIRO,10639
3,10639,CANJA FIV FINEST PORTEIRA AZUL,12,5893-AU,PENN-GATE FINEST-ET,10,AX117712,VALSA DA CINEL,11,LVRG574,...,31/01/2014,F,PENN-GATE FINEST-ET,VALSA DA CINEL,1/2HOL 1/2GIR,519,GIRO812244,LVRG574,10639-GIRO,10639
4,10639,BEATRIZ TEATRO DO MEU SONHO,12,5941-AU,TEATRO DA SILVANIA,11,EFC383,FABULA DO MEU SONHO,10,C-3722,...,12/05/2016,F,TEATRO DA SILVANIA,FABULA DO MEU SONHO,1/2HOL 1/2GIR,519,EFC383,GIRO1845621,10639-GIRO,10639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,8030,SAVANA PLANET FIV FZD,12,4961-U,ENSENADA TABOO PLANET-ET,10,AX125509,NINA JAGUAR,11,FAZP84,...,31/07/2013,F,ENSENADA TABOO PLANET-ET,NINA JAGUAR,1/2HOL 1/2GIR,519,212019,FAZP84,26363-ABCZ,8030
300,8030,SAVANA PLANET FIV FZD,12,4961-U,ENSENADA TABOO PLANET-ET,10,AX125509,NINA JAGUAR,11,FAZP84,...,31/07/2013,F,ENSENADA TABOO PLANET-ET,NINA JAGUAR,1/2HOL 1/2GIR,519,212019,FAZP84,4655-ABCZ,8030
301,2389,TEQUILA GENGIS KHAN FIV CAMPAN,12,5278-AZ,GENGIS KHAN DE BRASILIA,11,RRP6097,RHOELANDT 384 BETSY SPIRIT ROY,10,BX323094,...,16/02/2017,F,GENGIS KHAN DE BRASILIA,RHOELANDT 384 BETSY SPIRIT ROY,1/2HOL 1/2GIR,519,RRP6097,GIRO1054443,2389-GIRO,2389
302,2384,BATUCADA 5003 EXPANDER FIV JM,34,7184-BC,STANTONS EXPANDER-ET,10,AX149579,COROLA JM MONTE ALVERNE,12,AA-7520,...,01/12/2017,F,STANTONS EXPANDER-ET,COROLA JM MONTE ALVERNE,3/4HOL 1/4GIR,795,GIRO1857624,GIRO793074,2384-GIRO,2384


In [30]:
from datetime import datetime

#df_animais_sem_id_touro = df_animais_sem_id_touro.drop(['reb_giro', 'GSPAI', 'GSMAE', 'RGPAI', 'RGMAE','GSMAE', 'DNV', 'MNV', 'ANV', 'UF', 'OP', 'DP', 'MP', 'AP', 'DURLAC', 'LTOT', ''], axis=1)

# Adiciona coluna "id"
df_animais_com_ids = df_animais_com_ids.assign(id=None)
#Atribui valores de id_produto a id
df_animais_com_ids['id'] = df_animais_com_ids['id_produto']
# Adiciona coluna "abcz" com valores iguais a null
df_animais_com_ids = df_animais_com_ids.assign(abcz=None)
#renomeia RGVACA para registro
df_animais_com_ids = df_animais_com_ids.rename(columns={'RGVACA':'registro'})
#renomeia nomea para nome
df_animais_com_ids = df_animais_com_ids.rename(columns={'nomea':'nome'})
# Adiciona coluna "paternidade" com valores iguais a null
df_animais_com_ids = df_animais_com_ids.assign(paternidade=None)
# Adiciona coluna "ck_sire" com valores iguais a null
df_animais_com_ids = df_animais_com_ids.assign(ck_sire=None)
# Adiciona coluna "ck_dam" com valores iguais a null
df_animais_com_ids = df_animais_com_ids.assign(ck_dam=None)
# Adiciona coluna "gen_valido" com valores iguais a null
df_animais_com_ids = df_animais_com_ids.assign(gen_valido=None)
# Adiciona coluna "nasc_est" com valores da nasc
df_animais_com_ids = df_animais_com_ids.assign(nasc_est=None)
#Atribui os valores de nasc a nasc_est
df_animais_com_ids['nasc_est'] = df_animais_com_ids['nasc']
# Adiciona coluna "gr_psg" com valores da nasc
df_animais_com_ids = df_animais_com_ids.assign(gr_psg=None)
# Adiciona coluna "gr_ps" com valores da nasc
df_animais_com_ids = df_animais_com_ids.assign(gr_ps=None)
# Adiciona coluna "gr_tp" com valores da nasc
df_animais_com_ids = df_animais_com_ids.assign(gr_tp=None)
# Adiciona coluna "rebc2_cri" com valores da nasc
df_animais_com_ids = df_animais_com_ids.assign(rebc_cri=None)
#renomeia rebc_or para rebc_pr
df_animais_com_ids = df_animais_com_ids.rename(columns={'rebc_or':'rebc_pr'})
# Adiciona coluna "cat" com valores da nasc
df_animais_com_ids = df_animais_com_ids.assign(cat=None)
# Adiciona coluna "ficha_a" com valores da nasc
df_animais_com_ids = df_animais_com_ids.assign(ficha_a=None)
#renomeia IDANIMAL para idanimal
df_animais_com_ids = df_animais_com_ids.rename(columns={'IDANIMAL':'idanimal'})
# Adiciona coluna "origem" com valores da nasc
df_animais_com_ids = df_animais_com_ids.assign(origem='GIRO')
# Adiciona coluna "data_insercao" com valores da nasc
df_animais_com_ids = df_animais_com_ids.assign(data_insercao=None)
# Adiciona uma nova coluna com a data atual no formato "YYYY-MM-DD"
data_atual = datetime.today().strftime('%Y-%m-%d')
df_animais_com_ids['data_insercao'] = data_atual


tt = df_animais_com_ids[['id_produto', 'id', 'abcz', 'registro','nome','SEXO','paternidade','ck_sire','id_touro','id_vaca','ck_dam','gen_valido','nasc','nasc_est','cgen_a','gs','gr_psg','gr_ps','gr_tp','rebc_cri','rebc_pr','cat','ficha_a','cod_prod','idanimal','origem', 'data_insercao']]

tt


,id_produto,id,abcz,registro,nome,SEXO,paternidade,ck_sire,id_touro,id_vaca,...,gr_ps,gr_tp,rebc_cri,rebc_pr,cat,ficha_a,cod_prod,idanimal,origem,data_insercao
0,GIRO1588703,GIRO1588703,None,9592-AG,LANAA FIV AFTERSHOCK BON.BELEM,F,None,None,207142,ACFG1840,...,None,None,None,10639-GIRO,None,None,NaN,1588703,GIRO,2023-03-07
1,GIRO1544370,GIRO1544370,None,0949-AK,FLORENTINA FIV ATWOOD SAMPA,F,None,None,208574,ACFG1848,...,None,None,None,10639-GIRO,None,None,NaN,1544370,GIRO,2023-03-07
2,GIRO2678185,GIRO2678185,None,9419-BI,JGSA 1216 FIV GILCREST PORTEIR,F,None,None,GIRO1455287,ERD137,...,None,None,None,10639-GIRO,None,None,NaN,2678185,GIRO,2023-03-07
3,GIRO1550355,GIRO1550355,None,5893-AU,CANJA FIV FINEST PORTEIRA AZUL,F,None,None,GIRO812244,LVRG574,...,None,None,None,10639-GIRO,None,None,NaN,1550355,GIRO,2023-03-07
4,GIRO1845642,GIRO1845642,None,5941-AU,BEATRIZ TEATRO DO MEU SONHO,F,None,None,EFC383,GIRO1845621,...,None,None,None,10639-GIRO,None,None,NaN,1845642,GIRO,2023-03-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,GIRO1383169,GIRO1383169,None,4961-U,SAVANA PLANET FIV FZD,F,None,None,212019,FAZP84,...,None,None,None,26363-ABCZ,None,None,NaN,1383169,GIRO,2023-03-07
300,GIRO1383169,GIRO1383169,None,4961-U,SAVANA PLANET FIV FZD,F,None,None,212019,FAZP84,...,None,None,None,4655-ABCZ,None,None,NaN,1383169,GIRO,2023-03-07
301,GIRO2030237,GIRO2030237,None,5278-AZ,TEQUILA GENGIS KHAN FIV CAMPAN,F,None,None,RRP6097,GIRO1054443,...,None,None,None,2389-GIRO,None,None,NaN,2030237,GIRO,2023-03-07
302,GIRO2055307,GIRO2055307,None,7184-BC,BATUCADA 5003 EXPANDER FIV JM,F,None,None,GIRO1857624,GIRO793074,...,None,None,None,2384-GIRO,None,None,NaN,2055307,GIRO,2023-03-07
